In [1]:
pip install pymed

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [3]:
import nucleus_api, nucleus
import os
import csv
import json
import datetime
import time
from nucleus_api.rest import ApiException
from pprint import pprint
import numpy as np
from pathlib import Path

configuration = nucleus_api.Configuration()
# configuration.host = 'localhost:5000'
configuration.host = 'http://localhost:5000/'
configuration.api_key['x-api-key'] = 'VRgaGSWm'

api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [4]:
from pymed import PubMed

pubmed = PubMed(tool="NLPLahont", email="21tuong.dc@vinuni.edu.vn")
results = pubmed.query("(e[Title]) AND ([DOI])", max_results=50)
print(results)

In [5]:
dataset = 'dataset_test'

In [19]:
nucleus.Dataset.delete_dataset(dataset)
pubmed = PubMed(tool="NLPLahont", email="21tuong.dc@vinuni.edu.vn")
results = pubmed.query("(e[Title]) AND ([DOI])", max_results=500)
for article in results:
    # Print a JSON representation of the object
    # print(article.title)
    # print(article.abstract)
    # print(article.publication_date)
    newdocument = {}
    newdocument["title"] = article.title
    newdocument["time"] = article.publication_date.strftime("%Y-%m-%d")
    newdocument["content"] = article.abstract
    # print(jsonfile)
    payload = nucleus_api.Appendjsonparams(dataset=dataset,
                                       document=newdocument)
    try:
        api_response = api_instance.post_append_json_to_dataset(payload)
        print(api_response.result)
    except ApiException as e:
        api_error = json.loads(e.body)
        print('ERROR:', api_error['message'])


DEBUG: delete-dataset  default dataset_test 0
{'n_documents': 1, 'size': 1420}
{'n_documents': 2, 'size': 2364}
{'n_documents': 3, 'size': 1189}
{'n_documents': 4, 'size': 2196}
{'n_documents': 5, 'size': 969}
{'n_documents': 6, 'size': 449}
{'n_documents': 7, 'size': 1630}
{'n_documents': 8, 'size': 2342}
{'n_documents': 9, 'size': 1797}
{'n_documents': 10, 'size': 1319}
{'n_documents': 11, 'size': 2353}
{'n_documents': 12, 'size': 1126}
{'n_documents': 13, 'size': 1519}
{'n_documents': 14, 'size': 1916}
{'n_documents': 15, 'size': 1604}
{'n_documents': 16, 'size': 1674}
{'n_documents': 17, 'size': 201}
{'n_documents': 18, 'size': 103}
{'n_documents': 19, 'size': 1597}
ERROR: {'message': "document input must have 'content' field."}
{'n_documents': 20, 'size': 4218}
{'n_documents': 21, 'size': 629}
{'n_documents': 22, 'size': 1437}
{'n_documents': 23, 'size': 1703}
{'n_documents': 24, 'size': 450}
{'n_documents': 25, 'size': 1855}
{'n_documents': 26, 'size': 1189}
{'n_documents': 27, '

In [21]:
#compute topics:
payload = nucleus_api.Topics(dataset=dataset, num_topics=5)
api_response = api_instance.post_topic_api(payload) 
#create a json file topics.json containing only topic keywords, weights and strength
topics = [{'keywords':topic.keywords, 
           'keywords_weight':topic.keywords_weight, 
           'strength':topic.strength} for topic in api_response.result.topics]
print(json.dumps(topics, indent=4, ensure_ascii=False))
# with open('demo_topics.json', 'w') as outfile:
with open('/demo/demo_topics.json', 'w') as outfile:
    json.dump(topics, outfile, indent=4, ensure_ascii=False)

[
    {
        "keywords": "cigarette users;non smokers;current cigarette;tobacco cigarette;compared non;users nicotine;users dual;dual users",
        "keywords_weight": [
            "0.4641",
            "0.0826",
            "0.0915",
            "0.0909",
            "0.0944",
            "0.063",
            "0.0567",
            "0.0567"
        ],
        "strength": "0.2678"
    },
    {
        "keywords": "hepatitis virus;virus hev;public health;hev infection;viral hepatitis;acute hepatitis;health problem;common acute",
        "keywords_weight": [
            "0.0615",
            "0.0325",
            "0.0465",
            "0.3249",
            "0.2608",
            "0.1959",
            "0.0412",
            "0.0366"
        ],
        "strength": "0.2511"
    },
    {
        "keywords": "non daily;smoking cessation;cigarette smoking;long term;term cigarette;short term;quit non;cessation wave",
        "keywords_weight": [
            "0.1715",
            "0.1668",
   